In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../Data/SpotifyDataTrim.csv')
ids = df[['artist_name', 'track_name']]

In [4]:
df.head()

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Future,Life Is Good (feat. Drake),5yY9lUy8nbvjM1Uyo1Uqoc,96,0.676,0.609,2,-5.831,0,0.4810,0.0706,0.000000,0.1520,0.5080,142.037,237735,4
1,Eminem,Godzilla (feat. Juice WRLD),7FIWs0pqAYbP91WWM0vlTQ,95,0.808,0.745,10,-5.260,0,0.3420,0.1450,0.000000,0.2920,0.8290,165.995,210800,4
2,Billie Eilish,No Time To Die,73SpzrcaHk0RQPFP73vqVR,92,0.380,0.219,4,-13.273,0,0.0358,0.9170,0.010400,0.0827,0.0517,73.537,242265,4
3,The Weeknd,After Hours,4ejyZ4uJEXUURcgKp1kP8v,88,0.663,0.567,5,-5.941,0,0.0309,0.0938,0.009780,0.1240,0.1390,108.959,361947,4
4,Justin Bieber,Forever (feat. Post Malone & Clever),2ZlCGeK30BLRNSPC832pNZ,87,0.825,0.483,8,-5.746,0,0.0800,0.5380,0.000015,0.2040,0.8920,139.980,219933,4


In [5]:
from sklearn.neighbors import NearestNeighbors

In [6]:
df.columns[4:]

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature'],
      dtype='object')

In [6]:
df.iloc[2, 4:].to_numpy()

array([82, 0.733, 0.71, 5, -5.849, 0, 0.0292, 0.145, 0.115, 0.0956, 0.965,
       127.975, 239600, 4], dtype=object)

In [19]:
import sqlite3
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pandas as pd
def find_similar_song(track_id):
    #Instantiate Model
    knn = NearestNeighbors(n_neighbors=20)
    #Establish SQL connection
    conn = sqlite3.connect('spotify_tracks.sqlite3')
    #Establish SQL cursor
    curs = conn.cursor()
    #Query DB for numeric song attributes
    query = '''
    SELECT popularity, danceability, energy, key, loudness,
    mode, speechiness, acousticness, instrumentalness, liveness,
    valence, tempo, duration_ms, time_signature FROM SpotifyData
    '''
    curs.execute(query)
    #Store song attribute data as list of tuples
    tuples = curs.fetchall()
    #Fit tuples to df
    df = pd.DataFrame(tuples)
    #Fit model to numeric song attributes
    knn.fit(df)
    #Query DB for input's track ID
    query = f'''
    SELECT popularity, danceability, energy, key, loudness,
    mode, speechiness, acousticness, instrumentalness, liveness,
    valence, tempo, duration_ms, time_signature
    FROM SpotifyData
    WHERE track_id='{track_id}'
    '''
    curs.execute(query)
    obs = curs.fetchone()
    #Reshape song's attributes to list
    song = np.array([attr for attr in obs]).reshape(1, -1)
    neighbors = knn.kneighbors(song)
    new_obs = neighbors[1][0][5:20]
    #Query list of recommended songs and their attributes
    query = f'''
    SELECT * FROM SpotifyData
    WHERE id IN {tuple(new_obs)}
    '''
    curs.execute(query)
    recs = curs.fetchall()
    conn.close()
    return recs

In [20]:
find_similar_song('5yY9lUy8nbvjM1Uyo1Uqoc')

[(5023,
  'TWICE',
  'YES or YES',
  '3V23oYrFVc5VW1Ftq2wPCu',
  70,
  0.7020000000000001,
  0.9129999999999999,
  11,
  -2.977,
  1,
  0.0374,
  0.0514,
  3.71e-06,
  0.166,
  0.735,
  137.982,
  237685,
  4),
 (6472,
  'Rockwell',
  "Somebody's Watching Me - Single Version",
  '6A4Jc8npNo79BOgsrPptLA',
  66,
  0.767,
  0.7120000000000001,
  1,
  -4.128,
  0,
  0.0324,
  0.0409,
  8.08e-05,
  0.132,
  0.804,
  124.09100000000001,
  237760,
  4),
 (6518,
  'Katy Perry',
  'Chained To The Rhythm',
  '7qvxFz3JodM0A7xEM7k3YD',
  70,
  0.446,
  0.8,
  0,
  -5.404,
  1,
  0.174,
  0.0814,
  0.0,
  0.19899999999999998,
  0.474,
  189.89,
  237733,
  4),
 (7880,
  'Local Natives',
  'Coins',
  '53I7CviVHhNYEKth717RLG',
  59,
  0.5660000000000001,
  0.5760000000000001,
  5,
  -4.859,
  1,
  0.0365,
  0.0578,
  0.0,
  0.0523,
  0.607,
  138.045,
  237693,
  4),
 (9017,
  'Worn-Tin',
  'Sensitivity',
  '1F9WFVKQ0PT4EiZNoZQu2d',
  53,
  0.705,
  0.529,
  4,
  -6.585,
  1,
  0.028999999999999998,


In [35]:
conn.close()

NameError: name 'conn' is not defined